In [127]:
import kcf_api_calls
import os
import re
import pandas as pd
import outlook

In [128]:
def remove_condor_list(pumplist):
    for condor in condors:
        pumplist = list(filter(lambda a: a != condor, pumplist))
    return pumplist

condors = ['41', '44', '43', '45', '120', '121', '122', '134', '147',
           '149', '164', '181', '183', '185', '190', '189']

In [129]:
def find_condor(df, column):
    try:
        return int(re.search(r'''
                           >\s
                           (?P<pump>[0-9]+)
                           ?\s>
                            ''', df[column][0], re.X).group(1))
    except (TypeError, AttributeError) as e:
        return 0

In [130]:
def find_extra(df, column):
    try:
        return re.search(r'''
                        (?P<extra>>\s\w{2}\s\w+\s>\s)
                        ''', df[column][0], re.X).group(1)
    except (TypeError, AttributeError) as e:
        return 0

In [131]:
directory = r'C:\Users\austi\Desktop\React Nanodegree\MyReads Project\myreads\Daily_KCF_Report_Current\test'

for file in os.listdir(directory):
    with open(os.path.join(directory, file)) as stuff:
        data = stuff.read()
        pumplist = re.findall(r'''
                       >\s
                       (?P<pump>[0-9BTP-]+)
                       ?\s>
                        ''', data, re.X)

        holes = []

In [132]:
pumplist

['09',
 '79',
 '83',
 '83',
 '72',
 '43',
 '566',
 '684',
 '06',
 '333',
 '668',
 '52BTP-09006',
 '668',
 '284',
 '43',
 '06',
 '260',
 '393',
 '393',
 '104',
 '309',
 '104',
 '52BTP-09006',
 '120',
 '309',
 '120',
 '09',
 '333',
 '72',
 '260',
 '284',
 '566',
 '79',
 '684']

In [133]:
csv_file_path = os.path.join(directory, file)

In [134]:
kcf = pd.read_csv(csv_file_path, header=None, dtype=object)
with open(csv_file_path) as stuff:
    data = stuff.read()

    pumplist = remove_condor_list(pumplist)

    kcf.fillna(0, inplace=True)

    kcf = kcf.loc[:, (kcf != 0).any(axis=0)]
    columns = []

    for column in kcf.columns:
        if kcf[column][0] != 0:
            columns.append(column)
        else:
            continue
    to_delete = []

In [135]:
len(pumplist)

30

In [136]:
kcf.to_csv(r'C:\Users\austi\Desktop\React Nanodegree\MyReads Project\myreads\Daily_KCF_Report_Current\test\test.csv')

In [137]:
kcf[0][0]

'New Odessa Master > 09 > Fluid End > Hole 5 > Damage Accumulation'

In [138]:
for column in columns:
    for condor in condors:
        if int(condor) == find_condor(kcf, column):
            to_delete.append(column)
            to_delete.append(column + 1)
            to_delete.append(column + 2)
            to_delete.append(column + 3)

In [139]:
to_delete

[25, 26, 27, 28, 70, 71, 72, 73, 115, 116, 117, 118, 125, 126, 127, 128]

In [140]:
find_extra(kcf, column)

0

In [141]:
kcf.drop(labels=to_delete, axis=1, inplace=True)
for column in kcf.columns:
    try:
        holes.append(re.search(r'''
               (?P<hole>>\s\w+\s\d\s>)
                ''', kcf[column][0], re.X).group(1))
    except TypeError:
        pass

for condor in to_delete:
    if condor in columns:
        columns.remove(condor)




In [142]:
len(holes)

30

In [143]:
Hone = True
for pump, column, hole in zip(pumplist, columns, holes):

    if kcf[column][0] != 0:
        if Hone:
            kcf.loc[(1, column + 1)] = pump + " " + hole
            Hone = False
        else:
            kcf.loc[(1, column + 1)] = pump + " " + hole
            Hone = True
    kcf.drop(labels=[column + 2, column + 3], axis=1, inplace=True)
kcf.drop(labels=0, inplace=True)
kcf.set_index(kcf.astype(bool).sum(axis=0).idxmax(), inplace=True)
kcf.columns = kcf.iloc[0]
cols = [c for c in kcf.columns if c != 'Time (UTC)']

In [144]:
kcf

Time (UTC),Time (UTC),09 > Hole 5 >,Time (UTC),79 > Hole 1 >,Time (UTC),83 > Hole 1 >,Time (UTC),83 > Hole 5 >,Time (UTC),72 > Hole 1 >,...,Time (UTC),260 > Hole 5 >,Time (UTC),284 > Hole 1 >,Time (UTC),566 > Hole 1 >,Time (UTC),79 > Hole 5 >,Time (UTC),684 > Hole 1 >
95,,,,,,,,,,,,,,,,,,,,,
Time (UTC),Time (UTC),09 > Hole 5 >,Time (UTC),79 > Hole 1 >,Time (UTC),83 > Hole 1 >,Time (UTC),83 > Hole 5 >,Time (UTC),72 > Hole 1 >,...,Time (UTC),260 > Hole 5 >,Time (UTC),284 > Hole 1 >,Time (UTC),566 > Hole 1 >,Time (UTC),79 > Hole 5 >,Time (UTC),684 > Hole 1 >
9/18/2018 1:04:09 PM,9/18/2018 1:03:08 PM,0.5516475,9/18/2018 1:02:07 PM,0.8085819,9/18/2018 1:04:14 PM,1.140449,9/18/2018 1:03:19 PM,1.0107322,0,0,...,9/18/2018 6:23:59 PM,0.6158957,9/18/2018 1:03:15 PM,0.4515272,9/18/2018 1:03:08 PM,0.4884408,9/18/2018 1:02:07 PM,0.7146494,9/18/2018 1:04:18 PM,0.8596082
9/18/2018 1:22:07 PM,9/18/2018 1:04:06 PM,0.562825,9/18/2018 1:21:16 PM,0.9464935,9/18/2018 1:23:27 PM,1.1268724,9/18/2018 1:04:19 PM,0.7281582,0,0,...,9/18/2018 9:36:02 PM,0.4175159,9/18/2018 1:04:16 PM,0.7538068,9/18/2018 1:04:06 PM,0.8910469,9/18/2018 1:04:08 PM,0.8344598,9/18/2018 1:23:23 PM,0.9023609
9/18/2018 1:23:05 PM,9/18/2018 1:22:13 PM,0.3995193,9/18/2018 1:22:17 PM,0.7758463,9/18/2018 1:31:22 PM,1.2814984,9/18/2018 1:12:18 PM,1.3274356,0,0,...,9/18/2018 9:37:02 PM,0.4596148,9/18/2018 1:22:21 PM,0.4430753,9/18/2018 1:23:16 PM,0.4752359,9/18/2018 1:21:19 PM,0.7495436,9/18/2018 1:41:28 PM,1.7056503
9/18/2018 1:41:19 PM,9/18/2018 1:23:14 PM,0.3373059,9/18/2018 1:23:17 PM,0.953809,9/18/2018 1:39:17 PM,0.8687464,9/18/2018 1:31:48 PM,1.0628197,0,0,...,9/18/2018 9:45:05 PM,0.4715298,9/18/2018 1:23:18 PM,0.6523727,9/18/2018 1:59:37 PM,0.5972734,9/18/2018 1:22:21 PM,0.6869633,9/18/2018 1:42:28 PM,0.4867003
9/18/2018 1:42:20 PM,9/18/2018 1:30:17 PM,0.498184,9/18/2018 1:42:24 PM,0.5253396,9/18/2018 1:40:28 PM,1.3974813,9/18/2018 1:41:26 PM,1.0804706,0,0,...,9/18/2018 9:47:01 PM,0.4974789,9/18/2018 2:08:34 PM,0.6172421,9/18/2018 2:08:34 PM,0.5877065,9/18/2018 1:23:21 PM,0.7684738,9/18/2018 1:58:43 PM,0.4392122
9/18/2018 1:49:18 PM,9/18/2018 1:31:19 PM,0.4792286,9/18/2018 2:08:46 PM,0.5123025,9/18/2018 1:41:18 PM,0.7668334,9/18/2018 1:42:26 PM,0.4331442,0,0,...,9/18/2018 9:58:55 PM,0.4754645,9/18/2018 2:09:35 PM,0.5088642,9/18/2018 2:09:35 PM,0.5563895,9/18/2018 1:40:28 PM,0.6743266,9/18/2018 1:59:42 PM,0.400762
9/18/2018 1:50:28 PM,9/18/2018 1:41:23 PM,0.5181098,9/18/2018 2:09:46 PM,0.4747745,9/18/2018 1:42:19 PM,0.6108136,9/18/2018 1:50:28 PM,0.8296281,0,0,...,9/18/2018 9:59:56 PM,0.4528211,9/18/2018 2:10:35 PM,0.5067412,9/18/2018 2:10:35 PM,0.5712063,9/18/2018 1:41:20 PM,1.0153816,9/18/2018 2:11:44 PM,0.5233242
9/18/2018 2:09:34 PM,9/18/2018 1:42:24 PM,0.4938082,9/18/2018 2:10:47 PM,0.5126456,9/18/2018 1:50:28 PM,1.1368646,9/18/2018 1:59:41 PM,0.4081885,0,0,...,9/18/2018 10:00:56 PM,0.4714869,9/18/2018 2:11:35 PM,0.5174171,9/18/2018 2:11:35 PM,0.5740368,9/18/2018 1:42:21 PM,0.4149465,9/18/2018 2:20:17 PM,0.5000947


In [145]:
kcf_filtered = kcf[cols].drop('Time (UTC)')

In [146]:
kcf_filtered

Time (UTC),09 > Hole 5 >,79 > Hole 1 >,83 > Hole 1 >,83 > Hole 5 >,72 > Hole 1 >,566 > Hole 5 >,684 > Hole 5 >,06 > Hole 1 >,333 > Hole 1 >,668 > Hole 5 >,...,52BTP-09006 > Hole 1 >,309 > Hole 5 >,09 > Hole 1 >,333 > Hole 5 >,72 > Hole 5 >,260 > Hole 5 >,284 > Hole 1 >,566 > Hole 1 >,79 > Hole 5 >,684 > Hole 1 >
95,,,,,,,,,,,,,,,,,,,,,
9/18/2018 1:04:09 PM,0.5516475,0.8085819,1.140449,1.0107322,0,1.079995,1.2534265,0.5247755,1.1507986,0.3392721,...,0.5247755,1.1625028,0.5911039,2.80776,0,0.6158957,0.4515272,0.4884408,0.7146494,0.8596082
9/18/2018 1:22:07 PM,0.562825,0.9464935,1.1268724,0.7281582,0,0.5887955,1.4235518,0.5216912,1.0282185,0.3342759,...,0.5216912,1.1880695,0.6457755,1.3578624,0,0.4175159,0.7538068,0.8910469,0.8344598,0.9023609
9/18/2018 1:23:05 PM,0.3995193,0.7758463,1.2814984,1.3274356,0,1.0942722,1.1795782,0.5321483,1.5163411,0.3456692,...,0.5321483,0.8308539,0.483598,2.4206105,0,0.4596148,0.4430753,0.4752359,0.7495436,1.7056503
9/18/2018 1:41:19 PM,0.3373059,0.953809,0.8687464,1.0628197,0,0.5698618,0.7766505,0.5420686,2.792268,0.3328179,...,0.5420686,1.3253951,0.4244886,1.8699507,0,0.4715298,0.6523727,0.5972734,0.6869633,0.4867003
9/18/2018 1:42:20 PM,0.498184,0.5253396,1.3974813,1.0804706,0,0.6299332,0.4505517,7.0663898,0.5676284,0.4035678,...,7.0663898,2.0484711,0.427562,1.9099768,0,0.4974789,0.6172421,0.5877065,0.7684738,0.4392122
9/18/2018 1:49:18 PM,0.4792286,0.5123025,0.7668334,0.4331442,0,0.5636756,0.4359708,4.4331224,0.5198329,0.3817856,...,4.4331224,0.5389996,0.320106,2.2896964,0,0.4754645,0.5088642,0.5563895,0.6743266,0.400762
9/18/2018 1:50:28 PM,0.5181098,0.4747745,0.6108136,0.8296281,0,0.5757692,0.5010991,4.4332047,0.5571073,0.4159724,...,4.4332047,0.4633392,0.4493292,1.0816283,0,0.4528211,0.5067412,0.5712063,1.0153816,0.5233242
9/18/2018 2:09:34 PM,0.4938082,0.5126456,1.1368646,0.4081885,0,0.5884382,0.4947806,0.5593256,0.55373,0.4297742,...,0.5593256,0.4690822,0.5021209,1.9263446,0,0.4714869,0.5174171,0.5740368,0.4149465,0.5000947
9/18/2018 2:10:34 PM,0.4218767,0.4896877,0.5053946,0.4689855,0,0.6118249,0.5020711,0.5470641,0.5632613,0.3829043,...,0.5470641,0.4945954,0.4818905,0.5450278,0,0.4695356,0.5400523,0.5734436,0.9911408,0.5182995


In [147]:
kcf_fn = kcf_filtered.apply(pd.to_numeric)
kcf_fn = kcf_fn.loc[:, (kcf_fn != 0).any(axis=0)]

In [149]:
kcf_greater = kcf_fn[kcf_fn > .65]
unique_pump_list = []
for pump in pumplist:
    if pump not in unique_pump_list:
        unique_pump_list.append(pump)
kcf_average = pd.DataFrame(columns=unique_pump_list)
for unique_pump in unique_pump_list:
    i = 0
    for index in kcf_greater.mean(axis=0).index:
        if unique_pump == index[0:len(unique_pump)]:
            if i <= 1:
                kcf_average[unique_pump].loc[i] = kcf_greater.mean(axis=0)[index]
                print(kcf_average[unique_pump].loc[i])
                i += 1
                print(kcf_greater.mean(axis=0)[index])
                print(i)
kcf_final = pd.DataFrame()
for pump in kcf_average.columns:
    kcf_final[pump] = kcf_average[pump]
    largest = kcf_final.mean().nlargest(n=3).index.tolist()
    smallest = kcf_final.mean().nsmallest(n=3).index.tolist()
    averageDA = round(kcf_final.mean().mean(), 1)

1.5191288384615382
1.5191288384615382
1
1.1425647276470583
1.1425647276470583
2
0.9888916845161293
0.9888916845161293
1
0.8423099187499997
0.8423099187499997
2
1.429338450387597
1.429338450387597
1
1.3719879516746405
1.3719879516746405
2
0.9904919877358483
0.9904919877358483
1
0.8294755036144575
0.8294755036144575
2
2.0148597568181823
2.0148597568181823
1
2.081642674709976
2.081642674709976
2
3.342550268571428
3.342550268571428
1
1.5281762209677425
1.5281762209677425
2
1.3319960069637882
1.3319960069637882
1
2.0551866492537303
2.0551866492537303
2
0.9230337848708485
0.9230337848708485
1
1.073211833018868
1.073211833018868
2
1.5281762209677425
1.5281762209677425
1
3.342550268571428
3.342550268571428
2
1.3130742148337597
1.3130742148337597
1
0.9777672543046352
0.9777672543046352
2
0.8908391153846152
0.8908391153846152
1
1.3277305703125002
1.3277305703125002
2
1.8287137795212767
1.8287137795212767
1
1.3457505614325076
1.3457505614325076
2
1.154932444680851
1.154932444680851
1
1.1660120617

smallest

In [122]:
unique_pump_list

['09',
 '79',
 '83',
 '72',
 '566',
 '684',
 '06',
 '333',
 '668',
 '52BTP-09006',
 '284',
 '260',
 '393',
 '104',
 '309']

In [150]:
kcf_final

,09,79,83,72,566,684,06,333,668,52BTP-09006,284,260,393,104,309
0,1.519129,0.988892,1.429338,NaN,0.990492,2.014860,3.342550,1.331996,0.923034,1.528176,1.313074,0.890839,1.828714,1.154932,1.182352
1,1.142565,0.842310,1.371988,NaN,0.829476,2.081643,1.528176,2.055187,1.073212,3.342550,0.977767,1.327731,1.345751,1.166012,1.048358


In [118]:
unique_pump_list

['09',
 '79',
 '83',
 '72',
 '566',
 '684',
 '06',
 '333',
 '668',
 '52BTP-09006',
 '284',
 '260',
 '393',
 '104',
 '309']